# Credit Card Fraud Detection with a Simple Neural Network

This is a small addon project to another machine learning project I completed with standard classification models. 

By using tensorflow/keras I have created a simple neural network which takes 3 features from a credit card transaction dataset and classifies the transaction as fraudulent or not fraudulent.

This sort of model could be very useful for banks for fraud prevention and minimisation as it would allow them to identify fraudulent transactions quickly. Which would then enable them to either reverse the transactions or freeze the card to prevent further spending.

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

np.set_printoptions(suppress = True) # Stopping scientific notation in numpy outputs.

## Import the Data with Numpy

In [2]:
credit_arr = np.genfromtxt(fname = r"C:\Users\user1\Documents\Python Scripts\archive(3)\card_transdata.csv", 
                           delimiter = ",", 
                           skip_header = 1) # Skipping the column names as they'll just read NaN.

## Data Preprocessing

This dataset is already perfect, still some preprocessing needs to be done. Such as seperating the numerical and categorical features as well as feature scaling for the numerical features.

In [3]:
data = credit_arr.copy()

In [4]:
num_features = data[:, :3] # Features, numerical and categorical.
targets = data[:, -1] # Targets, categorical.

In [5]:
num_features

array([[57.87785658,  0.31114001,  1.94593998],
       [10.8299427 ,  0.1755915 ,  1.29421881],
       [ 5.09107949,  0.80515259,  0.42771456],
       ...,
       [ 2.91485699,  1.47268669,  0.21807549],
       [ 4.25872939,  0.24202337,  0.47582206],
       [58.10812496,  0.31811012,  0.38691985]])

Column 1: Distance from Home

Column 2: Distance from last Transaction

Column 3: Ratio to Median Purchase Price

In [6]:
scaler = MinMaxScaler()

scaler.fit(X = num_features)

MinMaxScaler()

In [7]:
features_scaled = scaler.transform(X = num_features)

In [8]:
features_scaled

array([[0.00544291, 0.00002624, 0.00725   ],
       [0.00101809, 0.00001481, 0.00481638],
       [0.00047835, 0.00006793, 0.00158072],
       ...,
       [0.00027368, 0.00012426, 0.0007979 ],
       [0.00040007, 0.00002041, 0.00176036],
       [0.00546457, 0.00002683, 0.00142839]])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(features_scaled, targets, test_size = 0.2)

## Constructing the Network

In [10]:
n_outputs = 2 # Rows in the output column vector (Yes/No).
n_hidden1 = 10 # Number of nodes in layer 1.
n_hidden2 = 10 # Number of nodes in layer 2.
n_input = 3

In [11]:
NN_model1 = keras.Sequential(layers = [keras.layers.Dense(units = n_hidden1, activation = "relu", input_dim = n_input), 
                                      keras.layers.Dense(units = n_hidden2, activation = "relu"), 
                                      keras.layers.Dense(units = n_outputs, activation = "softmax")])

In [12]:
NN_model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                40        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 22        
Total params: 172
Trainable params: 172
Non-trainable params: 0
_________________________________________________________________


In [13]:
NN_model1.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["Accuracy"])

In [14]:
n_epochs = 4

NN_model1.fit(x = x_train, y = y_train, epochs = n_epochs)

Epoch 1/4
25000/25000 [==============================] - 30s 1ms/step - loss: 0.1647 - Accuracy: 0.9260
Epoch 2/4
25000/25000 [==============================] - 28s 1ms/step - loss: 0.1355 - Accuracy: 0.9303
Epoch 3/4
25000/25000 [==============================] - 28s 1ms/step - loss: 0.1311 - Accuracy: 0.9313
Epoch 4/4
25000/25000 [==============================] - 29s 1ms/step - loss: 0.1225 - Accuracy: 0.9341


In [15]:
test_predict = x_train[10:12, :] # Testing some predictings from the training set.

NN_model1.predict(x = test_predict)

array([[0.76438046, 0.23561946],
       [1.        , 0.        ]], dtype=float32)

## On the Test Set

In [16]:
test_set_pred = NN_model1.predict(x = x_test)

In [17]:
fraud_predict_set = test_set_pred[:, 1]

mapping_set = [0 if n < 0.5 else 1 for n in fraud_predict_set] # Taking the softmax output of the fraudulent output node and mapping the values to 0 or 1.

In [18]:
y_final_predict = np.array(mapping_set)

In [19]:
acc_score = accuracy_score(y_true = y_test, y_pred = y_final_predict)

print("The accuracy score for this Neural Network model on the test set is: " + str(acc_score))

The accuracy score for this Neural Network model on the test set is: 0.936835


In [20]:
NN_conf_mat = confusion_matrix(y_true = y_test, y_pred = y_final_predict) # Confusion matrix for the model.

In [21]:
def recall_score(true_pos, false_neg):
    """Takes the true positive and false negative number as inputs and returns the recall score of the model."""
    
    rec_score = true_pos/(true_pos + false_neg)
    
    return rec_score

def precision_score(true_pos, false_pos):
    """Takes the true positive and false positve number as input and returns the precision score of the model."""
    
    prec_score = true_pos/(true_pos + false_pos)
    
    return prec_score

In [22]:
TP = NN_conf_mat[1, 1] # True Positive.
FN = NN_conf_mat[1, 0] # False Negative.
FP = NN_conf_mat[0, 1] # False Positive.

rec_score = recall_score(true_pos = TP, false_neg = FN)
prec_score = precision_score(true_pos = TP, false_pos = FP)

print(rec_score)
print(prec_score)

0.7113818823663384
0.6228936085352478


Higher recall score than precision score, in the context of credit fraud prevention I feel this is appropriate. This is because our main goal is to predict as many actually fraudulent transactions as possible in order minimize losses and consumer harm. If we have a lower precision (ie. predicting many transactions as fraudulent when a fair few aren't) then this doesnt matter so much as its only a slight nuisance to the consumer and can be fixed quickly with failsafes.

In [23]:
def f1_score(precision, recall):
    """Takes precision and recall values as inputs and returns the f1 score of the model."""
    
    f1 = 2*((precision*recall)/(precision + recall))
    
    return f1

In [24]:
f1 = f1_score(precision = prec_score, recall = rec_score)

print(f1)

0.6642035033624838


## Presenting Results

As can be observed from the evaluation metrics, a simple neural network does offer strong predictive power for classifying credit card transactions as either fraudulent or not fraudulent. This predictive power may even offer an advantage over standard machine learning models. I know in some contexts neural networks are considered "overkill" however if a model like this can save a bank lots of money by accurately classifying fraud then implementing it could be quite useful.